# Transpose Lower Triangular Matrix Vector Multiply Routines

This notebook walks you through how to implement $ x := L^T x $ where $ L $ is lower triangular.  Vector $ y $ is not to be touched (and, indeed, not even passed into the routines).  This is a little trickier than you might think, especially when you work with upper triangular matrix $ U $, later.  You may want to do a few small problems by hand if you don't get the right answer.  Also, PictureFLAME may help you see what is going on.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with the "import laff as laff" and "import flame" statements.

## Algorithm that takes dot products

## The routine <br> <code> Trmv_lt_unb_var1( L, x ) </code>

This routine, given lower triangular $ L \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := L^T x $.  The "_lt_" in the name of the routine indicates this is the "lower, transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
<li> <code> laff.scal( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [1]:
import flame
import laff as laff

def Trmv_lt_unb_var1(L, x):

    LTL, LTR, \
    LBL, LBR  = flame.part_2x2(L, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    while LTL.shape[0] < L.shape[0]:

        L00,  l01,      L02,  \
        l10t, lambda11, l12t, \
        L20,  l21,      L22   = flame.repart_2x2_to_3x3(LTL, LTR, \
                                                        LBL, LBR, \
                                                        1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.scal(lambda11, chi1)
        laff.dots(l21, x2, chi1)

        #------------------------------------------------------------#

        LTL, LTR, \
        LBL, LBR  = flame.cont_with_3x3_to_2x2(L00,  l01,      L02,  \
                                               l10t, lambda11, l12t, \
                                               L20,  l21,      L22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

    flame.merge_2x1(xT, \
                    xB, x)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [2]:
from numpy import random
from numpy import matrix

L = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
xold = matrix( random.rand( 4,1 ) )

# Notice that L is not lower triangular.  We will only use the lower triangular part.

print( 'L before =' )
print( L )

print( 'x before =' )
print( x )

L before =
[[0.95543616 0.13856029 0.95936756 0.04436628]
 [0.3772219  0.81504561 0.74469316 0.01217993]
 [0.26833922 0.18015569 0.95188269 0.38995703]
 [0.68588542 0.6643499  0.03535234 0.00955437]]
x before =
[[0.41322985]
 [0.82828765]
 [0.69020168]
 [0.76354137]]


In [3]:
import numpy as np
laff.copy( x, xold )   # save the original vector x

Trmv_lt_unb_var1( L, x )

print( 'x after =' )
print( x )

#np.tril makes the matrix lower triangular
print( 'x - ( np.transpose( np.tril( L ) ) * xold ) = ' )
print( x - ( np.transpose( np.tril( L ) ) * xold ) )

x after =
[[1.41617304]
 [1.3066946 ]
 [0.683984  ]
 [0.00729516]]
x - ( np.transpose( np.tril( L ) ) * xold ) = 
[[0.]
 [0.]
 [0.]
 [0.]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

## The routine <br> <code> Trmv_lt_unb_var2( L, x ) </code>

This routine, given lower triangular $ L \in \mathbb{R}^{n \times n} $ and $ x \in \mathbb{R}^n $ computes $ x := L^T x $.  The "_lt_" in the name of the routine indicates this is the "lower triangular, transpose" matrix-vector multiplication.  

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
<li> <code> laff.scal( alpha, x ) </code> which computes $ x := \alpha x $.  </li> (You will want to use this one to update $ \chi_1 $ if you want to use PictureFrame.)
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [4]:
def Trmv_lt_unb_var2(L, x):

    LTL, LTR, \
    LBL, LBR  = flame.part_2x2(L, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    while LTL.shape[0] < L.shape[0]:

        L00,  l01,      L02,  \
        l10t, lambda11, l12t, \
        L20,  l21,      L22   = flame.repart_2x2_to_3x3(LTL, LTR, \
                                                        LBL, LBR, \
                                                        1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.axpy(chi1, l10t, x0)
        laff.scal(lambda11, chi1)

        #------------------------------------------------------------#

        LTL, LTR, \
        LBL, LBR  = flame.cont_with_3x3_to_2x2(L00,  l01,      L02,  \
                                               l10t, lambda11, l12t, \
                                               L20,  l21,      L22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

    flame.merge_2x1(xT, \
                    xB, x)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [5]:
from numpy import random
from numpy import matrix

L = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
xold = matrix( random.rand( 4,1 ) )

# L is not lower triangular.  We will only use the lower triangular part.

print( 'L before =' )
print( L )

print( 'x before =' )
print( x )


L before =
[[0.12299553 0.22285834 0.63176829 0.9888338 ]
 [0.24805796 0.09239472 0.22382339 0.86690499]
 [0.82776104 0.51767687 0.24268149 0.37413915]
 [0.12362951 0.09094584 0.45004499 0.86752823]]
x before =
[[0.23730567]
 [0.20662316]
 [0.98687796]
 [0.39365877]]


In [6]:
laff.copy( x, xold )   # save the original vector y

Trmv_lt_unb_var2( L, x )

print( 'x after =' )
print( x )

#np.tril makes the matrix lower triangular
print( 'x - ( np.transpose( np.tril( L ) ) * xold ) = ' )
print( x - ( np.transpose( np.tril( L ) ) * xold ) )

x after =
[[0.94600902]
 [0.56577641]
 [0.41666117]
 [0.3415101 ]]
x - ( np.transpose( np.tril( L ) ) * xold ) = 
[[0.]
 [0.]
 [0.]
 [0.]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.